In [68]:
from importlib import reload

import DataProvider
import Betas

reload(DataProvider)
reload(Betas)

<module 'Betas' from '/home/vincent/vincent/sharedwork/data/market-hedging/Betas.py'>

# Compute betas

In [54]:
import configparser

CONFIG_NAME = "config.ini"
config = configparser.ConfigParser()
config.read(CONFIG_NAME)

API_KEY = config["TIINGO"]["API_KEY"]
VALUE_COL = config["TIINGO"]["VALUE_COL"]

In [45]:
import DataProvider

n = 300
codes = DataProvider.get_codes(n)

In [53]:
%%time

import pandas_datareader as pdr
df2 = pdr.get_data_tiingo(codes, api_key=API_KEY)

CPU times: user 19.9 s, sys: 974 ms, total: 20.9 s
Wall time: 1min 29s


In [49]:
df.to_hdf("store.h5", "table")

In [52]:
len(codes)

300

In [50]:
df_spy = pdr.get_data_tiingo("SPY", api_key=API_KEY)

In [60]:
import os
cwd = os.getcwd()
print(cwd)

/home/vincent/vincent/sharedwork/data/market-hedging


In [62]:
import pandas as pd
pd.read_hdf("store.h5", "table")

adjClose  adjHigh  adjLow  adjOpen  adjVolume  close  \
symbol date                                                               
NDN    2010-01-04     13.30  13.3600  13.100    13.12     550166  13.30   
       2010-01-05     13.35  13.4850  13.270    13.31     795839  13.35   
       2010-01-06     14.02  14.4500  13.520    13.52    1187682  14.02   
       2010-01-07     14.12  14.2100  13.870    14.11     490128  14.12   
       2010-01-08     13.72  14.0300  13.630    13.76     403262  13.72   
       2010-01-11     13.71  13.7600  13.500    13.75     360242  13.71   
       2010-01-12     13.69  13.8800  13.610    13.68     341697  13.69   
       2010-01-13     13.60  13.7600  13.500    13.75     465803  13.60   
       2010-01-14     13.54  13.7800  13.530    13.62     424294  13.54   
       2010-01-15     13.41  13.7000  13.320    13.54     474966  13.41   
       2010-01-19     13.44  13.5200  13.360    13.39     338987  13.44   
       2010-01-20     13.22  13.4300  13.190    13.32     325549  13.22   
       2010-01-21     13.13  13.3400  12.930    13.27     381024  13.13   
       2010-01-22     12.71  13.2700  12.640    13.08     495389  12.71   
       2010-01-25     12.77  12.8600  12.550    12.81     220592  12.77   
       2010-01-26     12.94  13.2390  12.700    12.75     513409  12.94   
       2010-01-27     12.99  13.0500  12.840    12.85     272312  12.99   
       2010-01-28     12.73  13.0300  12.610    12.99     251592  12.73   
       2010-01-29     13.04  13.3400  12.750    12.75     562736  13.04   
       2010-02-01     13.40  13.4000  12.790    13.05     605240  13.40   
       2010-02-02     13.30  13.4300  13.070    13.38     685087  13.30   
       2010-02-03     13.37  13.4700  13.210    13.29    1410063  13.37   
       2010-02-04     15.27  15.3300  13.350    13.41    3599677  15.27   
       2010-02-05     15.39  16.1200  15.100    15.16    1904746  15.39   
       2010-02-08     15.14  15.8300  15.130    15.40    1144225  15.14   
       2010-02-09     15.30  15.5000  15.180    15.30    1419580  15.30   
       2010-02-10     15.44  15.5100  15.000    15.19     986392  15.44   
       2010-02-11     15.89  16.0700  15.254    15.47     890202  15.89   
       2010-02-12     16.31  16.8100  15.690    15.75    1444303  16.31   
       2010-02-16     16.45  16.6200  16.210    16.37     885216  16.45   
...                     ...      ...     ...      ...        ...    ...   
SFL    2018-09-17     13.75  13.9000  13.600    13.90     644324  13.75   
       2018-09-18     13.65  13.8000  13.500    13.80     880812  13.65   
       2018-09-19     13.70  13.8000  13.600    13.70     644623  13.70   
       2018-09-20     13.80  13.8000  13.650    13.70     723453  13.80   
       2018-09-21     13.80  13.9000  13.750    13.75    1088308  13.80   
       2018-09-24     13.85  14.0250  13.800    13.80     493980  13.85   
       2018-09-25     13.90  14.0000  13.850    13.95     589152  13.90   
       2018-09-26     13.60  13.9500  13.550    13.90     506856  13.60   
       2018-09-27     13.75  14.0000  13.625    13.70     635035  13.75   
       2018-09-28     13.90  13.9500  13.750    13.75     617790  13.90   
       2018-10-01     13.97  14.0400  13.890    13.90     831880  13.97   
       2018-10-02     13.88  14.0400  13.850    13.99     581394  13.88   
       2018-10-03     14.03  14.0500  13.890    13.95     670936  14.03   
       2018-10-04     13.82  14.0900  13.780    14.04     728182  13.82   
       2018-10-05     13.83  13.9000  13.750    13.84     486595  13.83   
       2018-10-08     13.73  13.8200  13.600    13.80     462803  13.73   
       2018-10-09     13.68  13.7800  13.630    13.74     555131  13.68   
       2018-10-10     13.30  13.6800  13.280    13.64     819262  13.30   
       2018-10-11     13.14  13.3711  13.070    13.22    1109755  13.14   
       2018-10-12     13.18  13.2900  13.030    13.28    1050046  13.18   
       2018-10-15     13.37  13.4100  13.200

In [69]:
df = DataProvider.load_securities()

In [74]:
len(df.reset_index()["symbol"].unique())

300

In [51]:
%%time

start_date = "2013-10-01"
end_date = "2018-10-01"

n_stocks = 300
ref_name = "SPY"

# We take a random set of stocks
# TODO Ensure there's a certain proportion of small/medium/large stocks
data = DataProvider.get_random_securities(n_stocks, start_date, end_date)
data_ref = DataProvider.get_securities(ref_name, start_date, end_date)

KeyError: 'date'

In [ ]:
data.groupby("symbol").describe()

In [ ]:
df = Betas.attach_return(data, "symbol")
df_ref = Betas.attach_return(data_ref, "symbol")

df0 = df.reset_index().merge(
    df_ref.reset_index().drop("symbol", axis=1),
    on="date",
    suffixes=["", "_ref"]
).dropna().set_index(df.index.names)

In [ ]:
%%time
df = Betas.attach_beta(df0, "return_ref", "return", "symbol", 15, numpy=False)

In [ ]:
%%time
df = Betas.attach_beta(df0, "return_ref", "return", "symbol", 15, numpy=True)

In [ ]:
df.head(10)

For faster implementation, here are some alternative:

* Each regression can be computed in parallel. Dask is a library that implements numpy/pandas in parallel.
* Each security can be computed in parallel, and the least square solution for each security can be found by gradient descent using an exponentially decaying momentum in the gradient, by going through the points in a chronological order. This has the advantage that the data only has to be read once, and does not need to be loaded completely in memory, but this may affect the betas obtained.

Further development:

* Integrated code tests
* Validate and sanitize input
* Handle missing values
* Clean outliers
* Include standard errors on betas
* Non-parametric estimation of betas

# Analyse betas

In [ ]:
df.groupby("symbol")["beta"].describe()

There are suspicious outliers, when looking at the 25/75 percentile vs min/max.

In [ ]:
p = df.reset_index("symbol").groupby("symbol")["beta"].plot()

In [ ]:
p = df.reset_index("symbol").groupby("symbol")["beta"].plot(kind="density")

In [ ]:
threshold = 5
p = df.reset_index("symbol").groupby("symbol")["beta"].plot(kind="density", xlim=(-threshold, threshold))